In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from PIL import Image
import random

import os
import json
import cv2

import torch
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision import datasets, transforms

from transformers import TrOCRProcessor, VisionEncoderDecoderModel, default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments



Завиксируем всю случайность!

In [2]:
seed = 23
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.determenistic = True

Соберем наш датасет из полученных файлов

In [3]:
basketball_df = pd.read_csv('for_train\\train_bascetball.csv')
streetball_df = pd.read_csv('for_train\\train_streetball.csv')
volleyball_df = pd.read_csv('for_train\\train_volleyball.csv')

res_df = pd.concat([basketball_df, streetball_df, volleyball_df], axis=0)
res_df.shape

(150637, 3)

Разобьем датасет на train test и eval части

In [4]:
diff_df, test_df, = train_test_split(res_df, test_size=0.2, shuffle=True)
train_df, eval_df = train_test_split(diff_df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [5]:
store_pathes = []

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=3):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      
        file_name = self.df['file_name'][idx]
        text = str(self.df['text'][idx])
        store_pathes.append(file_name)
   
        image = Image.open(file_name).convert("RGB")
        image = image.resize((64, 64))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

Создадим наши датасеты

In [6]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=train_df,
                           processor=processor)
test_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=test_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=eval_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [7]:
print("Number of training examples:", len(train_dataset))
print("Number of testing examples:", len(test_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 96407
Number of testing examples: 30128
Number of validation examples: 24102


In [8]:
processor.batch_decode(eval_dataset[0]['labels'], skip_special_tokens=True)
#print(eval_dataset[0]['labels'])

['', '17', '']

Загрузим предъобученный трансформер

In [9]:
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
#model = VisionEncoderDecoderModel.from_pretrained("model")

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-stage1 and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = VisionEncoderDecoderModel.from_pretrained('model')
model.to(device)
print(device)

cuda


Сконфигурируем нашу модель

In [11]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Сконфигурируем цикл обучения

In [12]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=40,
    evaluation_strategy="steps",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    fp16=True, 
    output_dir="G:\\models1",
    logging_steps=1000,
    save_steps=20000,
    eval_steps=10000,
)

Оопределим метрику

In [13]:
import evaluate

cer_metric = evaluate.load("cer")
acc_metric = evaluate.load("accuracy")

In [14]:
'''
def compute_metrics(pred):
    label_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    #label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    #label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    x = [] 
    for j in pred_str:       
        if j.isdigit():
            x.append(int(j))
        else:
            x.append(1000)
    label_ids = [int(x) for x in label_ids]       
    acc = acc_metric.compute(predictions=x, references=label_ids)

    return {"accuracy": acc}

def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer
'''
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions    

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)    

    label_int = [int(x) for x in label_str]
    x = [] 
    for j in pred_str:       
        if j.isdigit():
            x.append(int(j))
        else:
            x.append(1000)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    acc = acc_metric.compute(predictions=x, references=label_int)

    return {"cer": cer,
            "accuracy": acc}


Переопределим оптимизатор

In [15]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters())

Запустим цикл обучения

In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

model.save_pretrained('model_next')

c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1285440 [00:00<?, ?it/s]

{'loss': 1.0801, 'learning_rate': 4.996118060741847e-05, 'epoch': 0.03}
{'loss': 0.6178, 'learning_rate': 4.9922361214836946e-05, 'epoch': 0.06}
{'loss': 0.4559, 'learning_rate': 4.9883502925068456e-05, 'epoch': 0.09}
{'loss': 0.4198, 'learning_rate': 4.984460573811302e-05, 'epoch': 0.12}
{'loss': 0.3682, 'learning_rate': 4.980574744834454e-05, 'epoch': 0.16}
{'loss': 0.3815, 'learning_rate': 4.97668502613891e-05, 'epoch': 0.19}
{'loss': 0.3271, 'learning_rate': 4.9727991971620615e-05, 'epoch': 0.22}
{'loss': 0.3205, 'learning_rate': 4.9689094784665175e-05, 'epoch': 0.25}
{'loss': 0.3494, 'learning_rate': 4.9650197597709734e-05, 'epoch': 0.28}
{'loss': 0.3146, 'learning_rate': 4.961133930794125e-05, 'epoch': 0.31}


c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


  0%|          | 0/8034 [00:00<?, ?it/s]

...

  0%|          | 0/8034 [00:00<?, ?it/s]

{'eval_loss': 3.769071578979492, 'eval_cer': 0.4029881154499151, 'eval_accuracy': {'accuracy': 0.5733963986391171}, 'eval_runtime': 4464.3624, 'eval_samples_per_second': 5.399, 'eval_steps_per_second': 1.8, 'epoch': 2.49}
{'loss': 0.0991, 'learning_rate': 4.685072815533981e-05, 'epoch': 2.52}
{'loss': 0.1053, 'learning_rate': 4.681183096838437e-05, 'epoch': 2.55}
{'loss': 0.1014, 'learning_rate': 4.6772972678615886e-05, 'epoch': 2.58}
{'loss': 0.1067, 'learning_rate': 4.6734075491660446e-05, 'epoch': 2.61}
{'loss': 0.1074, 'learning_rate': 4.669521720189196e-05, 'epoch': 2.65}
{'loss': 0.0905, 'learning_rate': 4.665632001493652e-05, 'epoch': 2.68}
{'loss': 0.1119, 'learning_rate': 4.661746172516804e-05, 'epoch': 2.71}
{'loss': 0.0971, 'learning_rate': 4.65785645382126e-05, 'epoch': 2.74}
{'loss': 0.119, 'learning_rate': 4.653966735125716e-05, 'epoch': 2.77}
{'loss': 0.1019, 'learning_rate': 4.6500770164301723e-05, 'epoch': 2.8}


  0%|          | 0/8034 [00:00<?, ?it/s]

{'eval_loss': 4.572350978851318, 'eval_cer': 0.3537181663837012, 'eval_accuracy': {'accuracy': 0.6240560949298813}, 'eval_runtime': 4531.9594, 'eval_samples_per_second': 5.318, 'eval_steps_per_second': 1.773, 'epoch': 2.8}
{'loss': 0.1087, 'learning_rate': 4.6461911874533234e-05, 'epoch': 2.83}
{'loss': 0.0965, 'learning_rate': 4.64230146875778e-05, 'epoch': 2.86}


KeyboardInterrupt: 

In [ ]:
url = "crops\\8\\ballerTV_137example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.cuda())

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

Определим eval_dataloader и Загрузим обученную модель

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=32)
model = VisionEncoderDecoderModel.from_pretrained('model')

# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_new_tokens = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Посчитаем точность обученной модели на eval датасете

In [ ]:
from sklearn.metrics import accuracy_score

torch.cuda.empty_cache()
# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
diff_acc = []
acc = []
i = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader):
    
        target_text = processor.batch_decode(batch['labels'], skip_special_tokens=True)
        target_text = [int(x) for x in target_text]
           
        pixel_values = batch['pixel_values']
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=4)       
       
        # Make a prediction
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

        x = [] 
        for j in generated_text:       
            if j.isdigit():
                x.append(int(j))
            else:
                x.append(1000)        
      
        bach_acc = accuracy_score(target_text, x)
        # Save the true and predicted labels
        diff_acc.append(bach_acc)  
        acc.append(bach_acc)

        if (i % 50  == 0) & (i > 2):
            accuracy = np.mean(diff_acc)
            print("Accuracy:", accuracy) 
        i += 1

    print(f"Total accuracy: {np.mean(acc)}")    

In [ ]:
df10 = pd.read_csv('anno_00new.csv')

eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=df10,
                           processor=processor)

print("Number of validation examples:", len(eval_dataset))

Посчитаем точность обученной модели на eval датасете

In [ ]:
from sklearn.metrics import accuracy_score

# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
y_true = []
y_pred = []
xx = []
with torch.no_grad():
    for i in tqdm(range(len(test_dataset)), nrows=2):
        url = test_df['file_name'][i]
        xx.append(url)
        target_text = int(test_df['text'][i])
        image = Image.open(url).convert("RGB")   

        pixel_values = processor(image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=3)

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        #pixel_values = batch['pixel_values']
        #generated_ids = model.generate(pixel_values.cuda())       

        # Make a prediction
        #generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
        x = []        
        if generated_text.isdigit():
            
            x = int(generated_text)
        else:
            x = 1000    
        
        # Save the true and predicted labels
        y_true.append(target_text)
        y_pred.append(x)

        if (i % 400  == 0) & (i > 2):
            accuracy = accuracy_score(y_true, y_pred)
            print("Accuracy:", accuracy) 

print("Accuracy:", accuracy)                   